# Day 6

In [3]:
import $ivy.`org.scalaz::scalaz-core:7.2.22`
import scalaz._
import Scalaz._

import $ivy.$                               

import scalaz._

import Scalaz._

## Writer? 中の人なんていません!

In [4]:
def isBigGang(x: Int): (Boolean, String) =
    (x > 9, "Compared gang size to 9.")

defined function isBigGang

In [5]:
implicit class PairOps[A](pair: (A, String)) {
    def applyLog[B](f: A => (B, String)): (B, String) = {
        val (x, log) = pair
        val (y, newlog) = f(x)
        (y, log ++ newlog)
    }
}

defined class PairOps

In [6]:
(3, "Smallish gang.") applyLog isBigGang

res5: (Boolean, String) = (false, "Smallish gang.Compared gang size to 9.")

In [7]:
implicit class PairOps[A, B: Monoid](pair: (A, B)) {
    def applyLog[C](f: A => (C, B)): (C, B) = {
        val (x, log) = pair
        val (y, newlog) = f(x)
        (y, log |+| newlog)
    }
}

defined class PairOps

In [8]:
(3, "Smallish gang.") applyLog isBigGang

res7: (Boolean, String) = (false, "Smallish gang.Compared gang size to 9.")

## Writer

```scala
type Writer[+W, +A] = WriterT[Id, W, A]
```

## WriterT

```scala
sealed trait WriterT[F[+_], +W, +A] { self =>
  val run: F[(W, A)]

  def written(implicit F: Functor[F]): F[W] =
    F.map(run)(_._1)
  def value(implicit F: Functor[F]): F[A] =
    F.map(run)(_._2)
}
```

In [9]:
3.set("Smallish gang.")

res8: Writer[String, Int] = WriterT(("Smallish gang.", 3))

In [10]:
3.set("something")

res9: Writer[String, Int] = WriterT(("something", 3))

In [11]:
"something".tell

res10: Writer[String, Unit] = WriterT(("something", ()))

In [11]:
MonadTell[Writer, String]

cmd11.sc:1: scalaz.Writer takes two type parameters, expected: one
val res11 = MonadTell[Writer, String]
                      ^

: 

In [11]:
MonadTell[Writer, String]

cmd11.sc:1: scalaz.Writer takes two type parameters, expected: one
val res11 = MonadTell[Writer, String]
                      ^

: 

## Writer に for 構文を使う

In [12]:
def logNumber(x: Int): Writer[List[String], Int] =
    x.set(List("Got number: " + x.shows))

defined function logNumber

In [13]:
def multWithLog: Writer[List[String], Int] = for {
    a <- logNumber(3)
    b <- logNumber(5)
} yield a * b

defined function multWithLog

In [14]:
multWithLog.run

res13: (List[String], Int) = (List("Got number: 3", "Got number: 5"), 15)

## プログラムにログを追加する

In [15]:
def gcd(a: Int, b: Int): Writer[List[String], Int] =
    if (b == 0) for {
        _ <- List("Finished with " + a.shows).tell
        } yield a
    else
        List(a.shows + " mod " + b.shows + " = " + (a % b).shows).tell >>= { _ =>
            gcd(b, a % b)
        }

defined function gcd

In [16]:
gcd(8, 3).run

res15: (List[String], Int) = (List("8 mod 3 = 2", "3 mod 2 = 1", "2 mod 1 = 0", "Finished with 1"), 1)

## 非効率な List の構築

In [17]:
Monoid[Vector[String]]

res16: Monoid[Vector[String]] = scalaz.std.VectorInstances$$anon$4@78cb01f7

In [19]:
def gcd(a: Int, b: Int): Writer[Vector[String], Int] =
    if (b == 0) for {
            _ <- Vector("Finished with " + a.shows).tell
        } yield a
    else for {
            result <- gcd(b, a % b)
            _ <- Vector(a.shows + " mod " + b.shows + " = " + (a % b).shows).tell
        } yield result

defined function gcd

In [20]:
gcd(8, 3).run

res19: (Vector[String], Int) = (Vector("Finished with 1", "2 mod 1 = 0", "3 mod 2 = 1", "8 mod 3 = 2"), 1)

## 性能の比較

In [21]:
def vectorFinalCountDown(x: Int): Writer[Vector[String], Unit] = {
  import annotation.tailrec
  @tailrec def doFinalCountDown(x: Int, w: Writer[Vector[String], Unit]): Writer[Vector[String], Unit] = x match {
    case 0 => w >>= { _ => Vector("0").tell }
    case x => doFinalCountDown(x - 1, w >>= { _ =>
      Vector(x.shows).tell
    })
  }
  val t0 = System.currentTimeMillis
  val r = doFinalCountDown(x, Vector[String]().tell)
  val t1 = System.currentTimeMillis
  r >>= { _ => Vector((t1 - t0).shows + " msec").tell }
}

def listFinalCountDown(x: Int): Writer[List[String], Unit] = {
  import annotation.tailrec
  @tailrec def doFinalCountDown(x: Int, w: Writer[List[String], Unit]): Writer[List[String], Unit] = x match {
    case 0 => w >>= { _ => List("0").tell }
    case x => doFinalCountDown(x - 1, w >>= { _ =>
      List(x.shows).tell
    })
  }
  val t0 = System.currentTimeMillis
  val r = doFinalCountDown(x, List[String]().tell)
  val t1 = System.currentTimeMillis
  r >>= { _ => List((t1 - t0).shows + " msec").tell }
}

defined function vectorFinalCountDown
defined function listFinalCountDown

In [23]:
val res = vectorFinalCountDown(10000).run

res: (Vector[String], Unit) = (
  Vector(
    "10000",
    "9999",
    "9998",
    "9997",
    "9996",
    "9995",
    "9994",
    "9993",
    "9992",
    "9991",
...

In [24]:
res._1.last

res23: String = "11 msec"

In [25]:
val res = listFinalCountDown(10000).run

res: (List[String], Unit) = (
  List(
    "10000",
    "9999",
    "9998",
    "9997",
    "9996",
    "9995",
    "9994",
    "9993",
    "9992",
    "9991",
...

In [26]:
res._1.last

res25: String = "2564 msec"

## Reader

In [27]:
val f = (_: Int) * 5

f: Int => Int = $sess.cmd26Wrapper$Helper$$Lambda$3435/592239235@68d7622e

In [29]:
val g = (_: Int) + 3

g: Int => Int = $sess.cmd28Wrapper$Helper$$Lambda$3446/1796858192@59f1b270

In [30]:
(g map f)(8)

res29: Int = 55

In [31]:
val f = ({(_: Int) * 2} |@| {(_: Int) + 10}) {_ + _}

f: Int => Int = scalaz.std.FunctionInstances$$anon$3$$Lambda$3453/109899854@598cedb7

In [32]:
f(3)

res31: Int = 19

In [33]:
val addStuff: Int => Int = for {
    a <- (_: Int) * 2
    b <- (_: Int) + 10
} yield a + b

addStuff: Int => Int = scalaz.std.FunctionInstances$$anon$3$$Lambda$3453/109899854@2ceee38d

In [34]:
addStuff(3)

res33: Int = 19